In [182]:
import pandas as pd
import numpy as np
import re
import json
import ast

In [183]:
df = pd.read_csv("../movies_dataset.csv")

C:\Users\Pablo\AppData\Local\Temp\ipykernel_4392\3976793484.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../movies_dataset.csv")


In [184]:
df.shape

(45466, 24)

In [185]:
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [186]:
#df_peliculas['release_date'] = pd.to_datetime(df_peliculas['release_date'], errors='coerce').dt.strftime('%Y-%m-%d')

In [187]:
df.isna().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

#### Tratamiento sobre columnas que sean colecciones

Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.

In [188]:
"""
coleccion = df['belongs_to_collection']
id = []
name = []
poster_path = []
backdrop_path = []
for linea in coleccion:
    try:
        lista_partes = linea.split(',')
        id.append(lista_partes[0][7:])
        name.append(lista_partes[1][10:-1])
        poster_path.append(lista_partes[2][18:-1])
        backdrop_path.append(lista_partes[3][20:-2])
        pass
    except:
        pass
dic = {'id':id[:-3],
       'name':name,
       'poster_path':poster_path,
       'backdrop_path':backdrop_path}
id = np.array(id[:-3])
name = np.array(name)
poster_path = np.array(poster_path)
backdrop_path = np.array(backdrop_path)
df_belongs_to_collection = pd.DataFrame(data=dic)
df_belongs_to_collection
"""

"\ncoleccion = df['belongs_to_collection']\nid = []\nname = []\nposter_path = []\nbackdrop_path = []\nfor linea in coleccion:\n    try:\n        lista_partes = linea.split(',')\n        id.append(lista_partes[0][7:])\n        name.append(lista_partes[1][10:-1])\n        poster_path.append(lista_partes[2][18:-1])\n        backdrop_path.append(lista_partes[3][20:-2])\n        pass\n    except:\n        pass\ndic = {'id':id[:-3],\n       'name':name,\n       'poster_path':poster_path,\n       'backdrop_path':backdrop_path}\nid = np.array(id[:-3])\nname = np.array(name)\nposter_path = np.array(poster_path)\nbackdrop_path = np.array(backdrop_path)\ndf_belongs_to_collection = pd.DataFrame(data=dic)\ndf_belongs_to_collection\n"

In [189]:
def convertir_colecciones(df, columnas):
    for column in columnas:
        df[column] = df[column].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])
        df = df.explode(column)
        col_df = pd.json_normalize(df[column])
        col_df = col_df.add_prefix(f'{column}')
        df = df.drop(columns=[column]).reset_index(drop=True).join(col_df)
    return(df)

columnas_colecciones = ['genres', 'production_companies', 'production_countries', 'spoken_languages','belongs_to_collection']
df = convertir_colecciones(df,columnas_colecciones)
df

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,vote_average,vote_count,genresid,genresname,production_companiesname,production_companiesid,production_countriesiso_3166_1,production_countriesname,spoken_languagesiso_639_1,spoken_languagesname
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,7.7,5415.0,16.0,Animation,Pixar Animation Studios,3.0,US,United States of America,en,English
1,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,7.7,5415.0,16.0,Animation,Pixar Animation Studios,3.0,US,United States of America,en,English
2,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,7.7,5415.0,16.0,Animation,Pixar Animation Studios,3.0,US,United States of America,en,English
3,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,7.7,5415.0,16.0,Animation,Pixar Animation Studios,3.0,US,United States of America,en,English
4,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,7.7,5415.0,35.0,Comedy,Pixar Animation Studios,3.0,US,United States of America,en,English
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582881,False,0,NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,...,3.8,6.0,28.0,Action,American World Pictures,6165.0,US,United States of America,en,English
582882,False,0,NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,...,3.8,6.0,18.0,Drama,American World Pictures,6165.0,US,United States of America,en,English
582883,False,0,NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,...,3.8,6.0,53.0,Thriller,American World Pictures,6165.0,US,United States of America,en,English
582884,False,0,NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,/aorBPO7ak8e8iJKT5OcqYxU3jlK.jpg,...,0.0,0.0,NaN,NaN,Yermoliev,88753.0,RU,Russia,NaN,NaN


In [190]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 582886 entries, 0 to 582885
Data columns (total 27 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   adult                           582886 non-null  object 
 1   budget                          582886 non-null  object 
 2   homepage                        191692 non-null  object 
 3   id                              582886 non-null  object 
 4   imdb_id                         582770 non-null  object 
 5   original_language               582790 non-null  object 
 6   original_title                  582886 non-null  object 
 7   overview                        580041 non-null  object 
 8   popularity                      582863 non-null  object 
 9   poster_path                     582297 non-null  object 
 10  release_date                    582680 non-null  object 
 11  revenue                         582859 non-null  float64
 12  runtime         

Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

In [191]:
df.dtypes

adult                              object
budget                             object
homepage                           object
id                                 object
imdb_id                            object
original_language                  object
original_title                     object
overview                           object
popularity                         object
poster_path                        object
release_date                       object
revenue                           float64
runtime                           float64
status                             object
tagline                            object
title                              object
video                              object
vote_average                      float64
vote_count                        float64
genresid                          float64
genresname                         object
production_companiesname           object
production_companiesid            float64
production_countriesiso_3166_1    

In [192]:
df['revenue'].fillna(0,inplace=True)
df['budget'].fillna(0,inplace=True)

Los valores nulos del campo release date deben eliminarse.

In [193]:
df = df.dropna(axis=0,subset='release_date')

De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

In [194]:
df['release_date'] = pd.to_datetime(df['release_date'],errors='coerce')
df['release_year'] = df['release_date'].dt.year

C:\Users\Pablo\AppData\Local\Temp\ipykernel_4392\2372018566.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['release_date'] = pd.to_datetime(df['release_date'],errors='coerce')
C:\Users\Pablo\AppData\Local\Temp\ipykernel_4392\2372018566.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['release_year'] = df['release_date'].dt.year


Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [195]:
df['budget'] = pd.to_numeric(df['budget'],errors='coerce')

C:\Users\Pablo\AppData\Local\Temp\ipykernel_4392\2291545301.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['budget'] = pd.to_numeric(df['budget'],errors='coerce')


In [196]:
df['roi'] = np.where((df['budget'].isna() | df['revenue'].isna() | (df['budget'] == 0)),
                     0, df['revenue'] / df['budget'])

C:\Users\Pablo\AppData\Local\Temp\ipykernel_4392\2840812579.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['roi'] = np.where((df['budget'].isna() | df['revenue'].isna() | (df['budget'] == 0)),


In [197]:
df['roi'].sample(3)

374068    0.000000
80677     4.048504
170457    0.522140
Name: roi, dtype: float64

In [198]:
df.columns

Index(['adult', 'budget', 'homepage', 'id', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'genresid', 'genresname',
       'production_companiesname', 'production_companiesid',
       'production_countriesiso_3166_1', 'production_countriesname',
       'spoken_languagesiso_639_1', 'spoken_languagesname', 'release_year',
       'roi'],
      dtype='object')

Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

In [199]:
df = df.drop(columns= ['video','imdb_id','adult','original_title','poster_path','homepage'])

#### segundo dataset

In [200]:
df_creditos = pd.read_csv("../credits.csv")

In [201]:
df_creditos

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


In [202]:

def expandcolumns(df, columns):
    for column in columns:
        df[column] = df[column].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else []) #Convierte el contenido de cada celda en la columna column de df de una cadena de texto que representa una lista de diccionarios a una lista de diccionarios Python, utilizando ast.literal_eval. Si el valor es nulo (pd.notnull(x)), se devuelve una lista vacía.
        # Expande la columna column en filas, de modo que cada elemento de la lista de diccionarios se convierte en una fila separada en df
        df = df.explode(column)
        # Normaliza la columna column después de expandirla, lo que significa que convierte cada diccionario dentro de la lista en una serie de columnas.
        col_df = pd.json_normalize(df[column])
        # Renombra las columnas normalizadas agregando el nombre original de la columna como prefijo, para evitar conflictos si hay nombres de columnas duplicados en diferentes listas de diccionarios.
        col_df = col_df.add_prefix(f'{column}')
        # Elimina la columna original column del DataFrame df y la reemplaza con las columnas normalizadas (col_df), asegurándose de que el índice del DataFrame se reinicie para evitar problemas con el índice de las filas.
        df = df.drop(columns=[column]).reset_index(drop=True).join(col_df)
    return df

columns_to_expand = ['cast', 'crew']


df_credit_norm = expandcolumns(df_creditos, columns_to_expand)

In [203]:
df_credit_norm

,id,castcast_id,castcharacter,castcredit_id,castgender,castid,castname,castorder,castprofile_path,crewcredit_id,crewdepartment,crewgender,crewid,crewjob,crewname,crewprofile_path
0,862,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,31.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg,52fe4284c3a36847f8024f49,Directing,2.0,7879.0,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
1,862,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,31.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg,52fe4284c3a36847f8024f4f,Writing,2.0,12891.0,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg
2,862,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,31.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg,52fe4284c3a36847f8024f55,Writing,2.0,7.0,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg
3,862,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,31.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg,52fe4284c3a36847f8024f5b,Writing,2.0,12892.0,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg
4,862,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,31.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg,52fe4284c3a36847f8024f61,Writing,0.0,12893.0,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9484663,227506,5.0,,52fe4ea59251416c7515d7e1,0.0,1261758.0,Aleksandr Chabrov,3.0,None,533bccebc3a36844cf0011a7,Directing,0.0,1085341.0,Director,Yakov Protazanov,/yyjbGdCs2ZN6IlZNCfmBWyuRDlt.jpg
9484664,227506,5.0,,52fe4ea59251416c7515d7e1,0.0,1261758.0,Aleksandr Chabrov,3.0,None,58ebbc26925141281908aa0a,Production,2.0,1195656.0,Producer,Joseph N. Ermolieff,None
9484665,227506,6.0,,52fe4ea59251416c7515d7e5,1.0,29199.0,Vera Orlova,4.0,/n1NXVGNzNxtqsMWxLT1h8GO8Kpi.jpg,533bccebc3a36844cf0011a7,Directing,0.0,1085341.0,Director,Yakov Protazanov,/yyjbGdCs2ZN6IlZNCfmBWyuRDlt.jpg
9484666,227506,6.0,,52fe4ea59251416c7515d7e5,1.0,29199.0,Vera Orlova,4.0,/n1NXVGNzNxtqsMWxLT1h8GO8Kpi.jpg,58ebbc26925141281908aa0a,Production,2.0,1195656.0,Producer,Joseph N. Ermolieff,None


#### leer el archivo y expandir columnas

In [204]:
df_creditos = pd.read_csv("../credits.csv")
print(df_creditos.shape)
print(df_creditos.columns)
df_creditos_cast = expandcolumns(df_creditos, ['cast'])
df_creditos_crew = expandcolumns(df_creditos, ['crew'])
print(df_creditos_crew.shape)
print(df_creditos_cast.shape)
df_creditos_cast = df_creditos_cast.drop(columns='crew')
df_creditos_crew = df_creditos_crew.drop(columns='cast')
df_creditos_cast

(45476, 3)
Index(['cast', 'crew', 'id'], dtype='object')
(465085, 9)
(564892, 10)


,id,castcast_id,castcharacter,castcredit_id,castgender,castid,castname,castorder,castprofile_path
0,862,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,31.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
1,862,15.0,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2.0,12898.0,Tim Allen,1.0,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg
2,862,16.0,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2.0,7167.0,Don Rickles,2.0,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg
3,862,17.0,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2.0,12899.0,Jim Varney,3.0,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg
4,862,18.0,Rex (voice),52fe4284c3a36847f8024fa5,2.0,12900.0,Wallace Shawn,4.0,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg
...,...,...,...,...,...,...,...,...,...
564887,227506,3.0,,52fe4ea59251416c7515d7d9,1.0,1090923.0,Nathalie Lissenko,1.0,None
564888,227506,4.0,,52fe4ea59251416c7515d7dd,2.0,1136422.0,Pavel Pavlov,2.0,None
564889,227506,5.0,,52fe4ea59251416c7515d7e1,0.0,1261758.0,Aleksandr Chabrov,3.0,None
564890,227506,6.0,,52fe4ea59251416c7515d7e5,1.0,29199.0,Vera Orlova,4.0,/n1NXVGNzNxtqsMWxLT1h8GO8Kpi.jpg


#### obtener un dataset con los valores unicos de genero

In [205]:
lis_dic = []
for x in df['genres']:
    valor_dic = ast.literal_eval(x)
    lis_dic.append(valor_dic)
    #diccionario = ast.literal_eval(diccionario_str)

df_generos = []
df_generos_id = []
for x in lis_dic:
    for y in x:
        if y['id'] not in df_generos_id:
            df_generos.append(y['name'])
            df_generos_id.append(y['id'])
        else:
            pass
dic_gen = {'id':df_generos_id,
           'genero':df_generos}
df_gen = pd.DataFrame(data=dic_gen)
df_gen

KeyError: 'genres'